<a href="https://colab.research.google.com/github/Zelechos/Generador-Css/blob/main/notebooks/Tests/Test_1(Scaled_dot_product_attention).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vamos ahora a resolver el problema utilizando el mecanismo de atención descrito anteriormente. Lo primero que tenemos que tener en cuenta es que los mecanismos de atención funcionan con conjuntos de vectores (secuencia), por lo que tenemos que reinterpretar nuestras imágenes. Para ello, vamos a dividirlas en 16 *patches* de 7x7. De esta manera, nuestras imágenes ahora serán secuencias de *patches* con las que nuestro mecanismo de atención podrá trabajar.

In [4]:
!git clone https://github.com/Zelechos/Generador-Css.git

fatal: destination path 'Generador-Css' already exists and is not an empty directory.


In [100]:
!pip install pytorch_lightning

In [5]:
import json

with open('/content/Generador-Css/dataset/TEST.json') as file:
    data = json.load(file)

#accedemos a la los datos
data_styles = data["styles"]
print(data_styles[0])


#accediendo a los estilos
styles = data_styles[0]["style"]
print("\nstyles access => ",styles)


#accediendo a los selectores
selector = data_styles[0]["selector"]
print("\nselector access => ",selector)

{'style': 'body { overflow : hidden ; background : rgb(25,35,125) } ', 'selector': 'body'}

styles access =>  body { overflow : hidden ; background : rgb(25,35,125) } 

selector access =>  body


In [26]:
data_css = []
styles = []
selectors = []
for data in data_styles:
  selectors.append(data["selector"])
  styles.append(data["style"])
 
# print(selectors)
# print(styles)
data_css.append(selectors)
data_css.append(styles)
print(data_css[0])
print(data_css[1])
print(len(data_css[0]))

['body', 'div.drop-container', 'div.drop', 'h1', '@font-face', 'body', '.logo', '.switch-left', '.pad-left', '.switch-right', '.pad-right', '.line', '.text', '.text', '.text', '.reload', '.logo', 'html,body', 'body', '*', '*', '*', '.css_lab', '.witch', 'body', 'body img', 'img', 'body h1', 'h1', 'body a', 'a', '@keyframes glow', 'from', 'to', 'body', 'h1', 'ul', 'li', 'li : hover span', 'li.invisible', 'li.animate', 'span', 'span : before', 'html', 'body', 'header', 'article', 'h3,section', 'h3', 'section', 'h3 : before', 'h3 : before,section : before', 'section : before', 'h3', 'section', 'h3 : before', 'section : before', 'p', 'a', 'body,html', 'body', 'html', 'div.codepen', 'div', '.codepen', '*', 'body', 'aside.context', '.context', 'aside', 'a', 'a : hover', 'footer', 'footer a', 'a', 'body', 'p', '.img-centered', '.bg-light-gray', '.bg-darkest-gray', '.btn-primary', '.navbar-default', '.navbar-default.navbar-brand', '.navbar-default.navbar-collapse', '.navbar-default.nav li a', 

In [7]:
import torch
import torch.nn as nn
import spacy # para la tokenizacion
import pandas as pd # para buscar en el archivo de anotaciones (atento)
from torch.nn.utils.rnn import pad_sequence #para el lote de almohadillas (pad batch) para que todos los datos tengan la misma longitud
from torch.utils.data import DataLoader , Dataset #para crea el dataset y el dataloader para el entrenamiento


device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [8]:
spacy_eng = spacy.load("en")
class Vocabulary:

  def __init__(self, freq_threshold):
    self.itos = {0:"<PAD>", 1:"<SOS>", 2:"<EOS>", 3:"<UNK>"}
    self.stoi = {"<PAD>":0, "<SOS>":1, "<EOS>":2, "<UNK>":3}
    self.freq_threshold = freq_threshold

  def __len__(self):
    #devolvemos la longitud de nuestro vocabulario
    return len(self.itos)

  #metodo para definir el ingles de nuestro vocabulario
  @staticmethod
  def tokenizer_eng(text):
    return [tok.text.lower() for tok in spacy_eng.tokenizer(text)]

  #ESTO ES LO QUE HACER EJEMPLO!!!
  # "body{background-color:red;}"" => ["body","{","background","-","color",":","red",";","}"]

  #construimos el vocabulario!!
  def build_vocabulary(self, sentence_list):
    frequencies = {}
    idx = 4

    for sentence in sentence_list:
      for word in self.tokenizer_eng(sentence):
        if word not in frequencies:
          frequencies[word] = 1
        else:
          frequencies[word] += 1

        if frequencies[word] == self.freq_threshold:
          self.stoi[word] = idx
          self.itos[idx] = word
          idx += 1

  def numericalize(self, text):
    tokenized_text = self.tokenizer_eng(text)
    return [
            self.stoi[token] if token in self.stoi else self.stoi["<UNK>"]
            for token in tokenized_text
    ]

In [122]:
class CssDataset(Dataset):

  def __init__ (self, css_styles, freq_threshold = 5):
    self.df = css_styles

    # vamos a obtener nuestros selectores
    self.selectors = self.df[0]

    # vamos a obtener nuestros styles
    self.styles = self.df[1]
    
    #Incializamos el Vocabulario (IMPORTANTEE!!!)
    self.vocab = Vocabulary(freq_threshold)
    self.vocab1 = Vocabulary(freq_threshold)

    self.vocab.build_vocabulary(self.styles) #Pasamos las Listas a el vocabulario
    self.vocab1.build_vocabulary(self.selectors) #Pasamos las Listas a el vocabulario
    

  def __len__(self):
    return len(self.df[1]) #devolvemos la longitud de nuestros dataset

  def __getitem__(self, index):
    selector = self.selectors[index]
    style = self.styles[index]

    numericalized_style = [self.vocab.stoi["<SOS>"]] 
    numericalized_style += self.vocab.numericalize(style) 
    numericalized_style.append(self.vocab.stoi["<EOS>"]) 

    numericalized_selector = [self.vocab1.stoi["<SOS>"]] 
    numericalized_selector += self.vocab1.numericalize(selector) 
    numericalized_selector.append(self.vocab1.stoi["<EOS>"]) 

    return torch.tensor(numericalized_selector) , torch.tensor(numericalized_style) 


train = CssDataset(data_css)
print('len =>',len(train))
# print(train.selectors[0])
# print(train.selectors[5])
# print(train.styles[0])
# print(train.styles[5])
# print(train.__getitem__(0))
# print(train.__getitem__(5))

len => 90


In [123]:
class MyCollate:

  def __init__(self, pad_idx):
    self.pad_idx = pad_idx

  def __call__(self, batch):
    targets = [item[0] for item in batch]
    targets = pad_sequence(targets, batch_first = False, padding_value = self.pad_idx )

    targets1 = [item[1] for item in batch]
    targets1 = pad_sequence(targets1, batch_first = False, padding_value = self.pad_idx )

    return targets, targets1


def get_loader(css_styles, batch_size = 20, num_workers = 8, shuffle = True, pin_memory = True):
  dataset = CssDataset(css_styles)
  pad_idx = dataset.vocab.stoi["<PAD>"]

  dataloader = DataLoader(
      dataset = dataset,
      batch_size = batch_size,
      num_workers = num_workers,
      shuffle= shuffle,
      pin_memory=pin_memory,
      collate_fn=MyCollate(pad_idx=pad_idx),
  )

  return dataset, dataloader

styles_clone = []
def main():
  dataset, dataloader = get_loader(data_css)
  print(dataloader)
  print(dataset)
  
  for idx , (selectors, styles) in enumerate(dataloader):
    print("selectors.shape")
    print(selectors.shape)
    print("styles.shape")
    print(styles.shape)
    # styles_clone.append(styles)
    # print(selectors)
    # print(styles)
    # print(type(selectors))
    # print(type(styles))
    # print(idx)

if __name__ == "__main__" :
  main()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


selectors.shape
torch.Size([7, 20])
styles.shape
torch.Size([78, 20])
selectors.shape
torch.Size([5, 20])
styles.shape
torch.Size([78, 20])
selectors.shape
torch.Size([9, 20])
styles.shape
torch.Size([73, 20])
selectors.shape
torch.Size([5, 20])
styles.shape
torch.Size([78, 20])
selectors.shape
torch.Size([7, 10])
styles.shape
torch.Size([69, 10])


In [124]:
dataset, dataloader = get_loader(data_css)


styles, selectors = next(iter(dataloader))
styles.shape, selectors.shape

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


(torch.Size([7, 20]), torch.Size([78, 20]))

In [117]:
import pytorch_lightning as pl

pl.__version__

'1.5.6'

In [118]:
import pytorch_lightning as pl
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.datasets import fetch_openml
import numpy as np
from torch.utils.data import DataLoader

In [125]:
# basado en: https://github.com/karpathy/minGPT/blob/master/mingpt/model.py
import math

class ScaledDotSelfAttention(torch.nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        
        # key, query, value projections
        self.key = torch.nn.Linear(n_embd, n_embd)
        self.query = torch.nn.Linear(n_embd, n_embd)
        self.value = torch.nn.Linear(n_embd, n_embd)

    def forward(self, x):
        B, L, F = x.size()
        
        # calculate query, key, values 
        k = self.key(x) # (B, L, F)
        q = self.query(x) # (B, L, F)
        v = self.value(x) # (B, L, F)
        
        # attention (B, L, F) x (B, F, L) -> (B, L, L)
        att = (q @ k.transpose(1, 2)) * (1.0 / math.sqrt(k.size(-1)))
        att = torch.nn.functional.softmax(att, dim=-1)
        y = att @ v # (B, L, L) x (B, L, F) -> (B, L, F)
        
        return y

class Model(MLP):

    def __init__(self, n_embd=20, seq_len=4*4):
        super().__init__()
        self.mlp = None

        self.attn = ScaledDotSelfAttention(n_embd)
        # self.actn = torch.nn.ReLU(inplace=True)
        self.fc = torch.nn.Linear(n_embd*seq_len, 10)

    def forward(self, x):
        x = self.attn(x)
        #print(x.shape)
        y = self.fc(x[:,-1,:])
        #print(y.shape)
        return y


In [128]:
class MLP(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(20, 20),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(20, 7)
        )

    def forward(self, x):
        return self.mlp(x)

    def predict(self, x):
        with torch.no_grad():
          y_hat = self(x)
          return torch.argmax(y_hat, axis=1)

    def compute_loss_and_acc(self, batch):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = (torch.argmax(y_hat, axis=1) == y).sum().item() / y.shape[0]
        return loss, acc

    def training_step(self, batch, batch_idx):
        loss, acc = self.compute_loss_and_acc(batch)
        self.log('loss', loss)
        self.log('acc', acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, acc = self.compute_loss_and_acc(batch)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.003)
        return optimizer

Nuestro modelo basado en atención tiene muchos menos parámetros. Esto es debido a que en el MLP todos las neuronas en la capa oculta están conectadas a todos los pixeles de la imagen. Ahora, sin embargo, reutilizamos conexiones a nivel de *patch*, de manera similar a cómo funcionan las redes convolucionales. 

In [129]:
mlp = MLP()
trainer = pl.Trainer(max_epochs=5)
trainer.fit(mlp, dataloader)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1580: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`."
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:122: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")

  | Name | Type       | Params
------------------------------------
0 | mlp  | Sequential | 567   
------------------------------------
567       Trainable params
0         Non-trainable params
567       Total params
0.002     Total estimated model params size (MB)
/usr/local/lib/python3.7/dis

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


RuntimeError: ignored